# LLMS-Generator Experiments

For experimentation with the LLMS-Generator Agent.

## Setup

To run this notebook, ensure you've authenticated to Google Cloud and installed the project's dependencies.

To setup the Google Cloud environment:

```bash
source scripts/setup-env.sh
```

Then to install the package dependencies into the virtual environment, use the `uv` tool:

1. From your agent's root directory, run `make install` to set up the virtual environment (`.venv`).
2. In this Jupyter notebook, select the kernel from the `.venv` folder to ensure all dependencies are available.

### Import Libraries

In [1]:
import os

import vertexai
from dotenv import load_dotenv
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.auth import default
from google.genai.types import Content, Part
from IPython.display import Markdown, display

### Load Environment Variables

In [2]:
dotenv_path = os.path.abspath('../.env')

if os.path.exists(dotenv_path):
    print(f"Loading environment variables from: {dotenv_path}")
    load_dotenv(dotenv_path=dotenv_path)
else:
    print(f"Warning: .env file not found at {dotenv_path}")

staging_project_id = os.getenv("GOOGLE_CLOUD_STAGING_PROJECT")
if staging_project_id:
    os.environ["GOOGLE_CLOUD_PROJECT"] = staging_project_id
    print(f"Set GOOGLE_CLOUD_PROJECT environment variable to: {staging_project_id}")


Loading environment variables from: /home/darren/localdev/Python/llms-gen/.env
Set GOOGLE_CLOUD_PROJECT environment variable to: scratch-dev-428715


### Ensure We're Authenticating to the Right Project

In [3]:
credentials, project_id = default()  # To use ADC
vertexai.init(project=staging_project_id, location="europe-west4", credentials=credentials) # type: ignore

## Running the Agent

In [4]:
from llms_gen_agent.agent import root_agent

09:15:27.790:llms_gen_agent - INFO: Logger initialised for llms_gen_agent.
09:15:27.790:llms_gen_agent - DEBUG: DEBUG level logging enabled.
09:15:28.426:llms_gen_agent - DEBUG: agent_name set to llms_gen_agent
09:15:28.427:llms_gen_agent - DEBUG: project_id set to scratch-dev-428715
09:15:28.427:llms_gen_agent - DEBUG: location set to global
09:15:28.427:llms_gen_agent - DEBUG: model set to gemini-2.5-flash
09:15:28.428:llms_gen_agent - DEBUG: genai_use_vertexai set to True


In [5]:
# Session and Runner
APP_NAME = "generate_llms_client"
USER_ID="test_user"
SESSION_ID="test_session"
TEST_REPO_PATH = "/home/darren/localdev/gcp/adk-docs" # You can change this to any repository you want to test

async def setup_session_and_runner():
    session_service = InMemorySessionService()
    session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)
    return session, runner

In [ ]:
# Agent Interaction
async def call_agent_async(query):
    content = Content(role='user', parts=[Part(text=query)])
    _, runner = await setup_session_and_runner()
    events = runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    final_response_content = "Final response not yet received."
    
    async for event in events:
        if function_calls := event.get_function_calls():
            tool_name = function_calls[0].name
            display(Markdown(f"_Using tool {tool_name}..._"))
        elif event.actions and event.actions.transfer_to_agent:
            personality_name = event.actions.transfer_to_agent
            display(Markdown(f"_Delegating to agent: {personality_name}..._"))
        elif event.is_final_response() and event.content and event.content.parts:
            final_response_content = event.content.parts[0].text

        # For debugging, print the raw type and content to the console
        print(f"DEBUG: Full Event: {event}")
    
    display(Markdown("## Final Message"))
    display(Markdown(final_response_content))

In [7]:
query = f"Generate the llms.txt file for the repository at {TEST_REPO_PATH}"
await call_agent_async(query)

_Using tool discover_files..._

09:15:30.184:llms_gen_agent - DEBUG: Entering tool: discover_files with repo_path: /home/darren/localdev/gcp/adk-docs
09:15:30.187:llms_gen_agent - DEBUG: Dirs
:/home/darren/localdev/gcp/adk-docs
/home/darren/localdev/gcp/adk-docs/docs
/home/darren/localdev/gcp/adk-docs/docs/deploy
/home/darren/localdev/gcp/adk-docs/docs/sessions
/home/darren/localdev/gcp/adk-docs/docs/tutorials
/home/darren/localdev/gcp/adk-docs/docs/streaming
/home/darren/localdev/gcp/adk-docs/docs/streaming/dev-guide
/home/darren/localdev/gcp/adk-docs/docs/callbacks
/home/darren/localdev/gcp/adk-docs/docs/context
/home/darren/localdev/gcp/adk-docs/docs/a2a
/home/darren/localdev/gcp/adk-docs/docs/runtime
/home/darren/localdev/gcp/adk-docs/docs/safety
/home/darren/localdev/gcp/adk-docs/docs/observability
/home/darren/localdev/gcp/adk-docs/docs/events
/home/darren/localdev/gcp/adk-docs/docs/tools
/home/darren/localdev/gcp/adk-docs/docs/mcp
/home/darren/localdev/gcp/adk-docs/docs/evaluate
/home/darren/localdev/gcp/adk-d

DEBUG: Full Event: content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'repo_path': '/home/darren/localdev/gcp/adk-docs'
        },
        id='adk-e49d33e3-2b04-4a07-84bd-7c4195509881',
        name='discover_files'
      ),
      thought_signature=b'\n\xf7\x04\x01\x1f\xcc\x85\xb6"R\x01\xbag\xd6\x94\x0f5\xefJ\x8e\xcc\xc2I\xf2U \x93\xaa\xa3~S\xc0P\xc9\xaa\t\xc5\xacF{`\\\x93e\xc1z\xcd\xc2\\\x1f\xf8\x9f\xc2\xa3_\\\xee\xefY\xb5\xfd\xd8f\x83\xfa\xf0]\xb8\xf4\xdd\x990\xc0\x8b\xa7\x9c\xb2%"U\xd9\x12.c\xbej\xa6\xad\xe0\x0f{4\xa8\nN\x1cP...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=21,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=21
    ),
  ],
  

_Using tool document_summariser_agent..._

DEBUG: Full Event: content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'request': ''
        },
        id='adk-906d4794-8f55-4c83-8f6e-47dd1cd79b26',
        name='document_summariser_agent'
      ),
      thought_signature=b'\n\xba\x02\x01\x1f\xcc\x85\xb6\xf2\xe3\x12}\xe1\x98/\x93G\xe8\x87\x82\xca\x03\x18\x96\x99\x99Y\xaa\x03r\x7fB>?\xf2\xbe\xa0^\xf5\x89N\xd4\xa2\xc5\xc1\xc5 \x93\xf1{\x9b\xc8\xa3\xb3\x19I\xe2U"\xca\xcd\xfcalJ\x01G;\x06\xc8"N\x99\xff\x83\xd9\xb1\x8f.\xe6\xaf\x84}\t\xb0\xec,\x1fW\xe1\x1a\xea\xdeI)\xb1\xc7...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=7,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=7
    ),
  ],
  prompt_tok

...

09:15:39.137:llms_gen_agent - DEBUG: Entering generate_llms_txt for repo_path: /home/darren/localdev/gcp/adk-docs
09:15:39.137:llms_gen_agent - DEBUG: We have 31 directories.
09:15:39.137:llms_gen_agent - DEBUG: We have 83 files
09:15:39.138:llms_gen_agent - DEBUG: We have 5 sumamries
09:15:39.138:llms_gen_agent - DEBUG: Project summary: The Agent Development Kit (ADK) is an open-source, code-first framework designed for building, evaluating, and deploying AI agents, offering flexibility and control across various platforms. It is model-agnostic and deployment-agnostic, optimized for the Google ecosystem but compatible with other frameworks, and supports both Python and Java development. ADK emphasizes features like a rich tool ecosystem, modular multi-agent systems, built-in observability, and easy deployment to facilitate complex AI agent development. 

This project's documentation outlines the core framework, installation methods, key features like flexible orchestration, multi-agen

DEBUG: Full Event: content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'repo_path': '/home/darren/localdev/gcp/adk-docs'
        },
        id='adk-dc7cc64b-39c3-40fe-aa89-b888fe4a8ccd',
        name='generate_llms_txt'
      ),
      thought_signature=b'\n\xfc\x01\x01\x1f\xcc\x85\xb6b\xde\xf4S\xf9T\xfd\xd1\xe2\x9d\x0c\xaf\tW\xb5]}|\x18\xd7\xf0\x12\x1c\xd9\x001\xd9V\xfb\xc3\xff\xbe\x0b\xc0\xd55\x98\xf0\xcdp\x8d\xfb=\x82i\xe8\x02\xb0\xa1%\x7f\x93\x11\xd9\x8a\xb5a#Z\xe9\x93\x04\x80\x82\xf8\xc3I\xec\xe6\xf0Cy2V\x89\xd1\xfc\xf7RY5X\x0fD\xab\xea\xf8\xe8 ...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=24,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_cou

### Final Message

The `llms.txt` file has been successfully generated at `/home/darren/localdev/Python/llms-gen/notebooks/temp/llms.txt`.

DEBUG: Full Event: content=Content(
  parts=[
    Part(
      text='The `llms.txt` file has been successfully generated at `/home/darren/localdev/Python/llms-gen/notebooks/temp/llms.txt`.',
      thought_signature=b'\n\xbc\x01\x01\x1f\xcc\x85\xb6\xfa\xd0\x7f\x1e\x98\xbaO\x8e\x97\x1b\xc7\xac\xcc$n&s\xed\xddU\xa5q\xc3\xd9\x87|\x1fi\n\xa4Z\x1d\x9d*\xe7\x1d\xe6\x00\xe3\xf5S\xc0#D\xb9\x92G0\x9fk2\'\xc2\xa3\x94+5\x9e\xaf\x15\xe3\xcfX\xbf\xed\xa7\xbc\xfdY\x1a\xb6\xa6"\xefo/\'\x9c\xd1\xdb\x14d8{\x9f)yKw...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  cache_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=3174
    ),
  ],
  cached_content_token_count=3174,
  candidates_token_count=39,
  candidates_tokens_details=[
    ModalityTokenCo